In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from Chemometrics.basic import psave,pload
import opusFC #读取IR数据
import re
import graphviz
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
%matplotlib inline

In [3]:
#IR
avgIRdata = pload('tw20_IRavg.p')
#mass
tw20 = pload('tw20.p')
raw_intensity = tw20.loc[:,'170627TW20_L1':'170627TW20_A2']
pct_intensity = tw20.loc[:,'170627TW20_L1.1':]
rt_mz_charge = tw20.loc[:,'Retention time (min)':'Charge']
rest = tw20.loc[:,:'Adducts']
rest.head()

,Compound name,Accepted Description,Retention time (min),m/z,Charge,Adducts
Compound ID,,,,,,
1,聚氧乙烯山梨醇酐（PS）,Tween20_PS_14,18.406033,798.470106,1,M+NH4
2,聚氧乙烯山梨醇酐（PS）,Tween20_PS_15,18.990433,842.495603,1,"M+NH4, M+2NH4"
3,聚氧乙烯山梨醇酐（PS）,Tween20_PS_16,19.546250,886.521321,1,"M+H, M+2H, M+NH4, M+2NH4"
4,聚氧乙烯山梨醇酐（PS）,Tween20_PS_17,20.152067,930.547731,1,"M+NH4, M+Na"
5,聚氧乙烯山梨醇酐（PS）,Tween20_PS_18,20.701467,974.573679,1,"M+NH4, M+Na"


In [4]:
def substr1(text,regexp):
    m = re.search(regexp, text)
    if m:
        return m.group(0)
l = pct_intensity.columns.tolist()
regexp = '(?<=_).+(?=.1$)'
pct_intensity.columns = [substr1(i,regexp) for i in l]

# 质谱数据建模

In [6]:
labels = [i[0] for i in pct_intensity.columns.tolist()]
labels

['L', 'G', 'B', 'B', 'A', 'A', 'A', 'A', 'A']

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import cross_val_score

In [8]:
clf = DecisionTreeClassifier(random_state=1)
X = pct_intensity.T
#scores = cross_val_score(clf,X,Y,cv=4) 交叉验证
clf = clf.fit(X,labels)
feature_names = tw20.loc[:,'Accepted Description'].tolist()#list
dot_data = export_graphviz(clf, out_file=None,
                          feature_names=feature_names,#list
                          class_names=clf.classes_,#list
                          filled=True, rounded=True,
                          special_characters=True)
graph = graphviz.Source(dot_data) 

~~~
mask = tw20.loc[:,'Accepted Description']=='Tween20_PI_6'
pct_intensity.loc[mask].T.iloc[:,0].sort_values()
~~~

~~~
export_graphviz(clf, out_file='tree.dot',
                          feature_names=feature_names,#list
                          class_names=clf.classes_,#list
                          filled=True, rounded=True,
                          special_characters=True)
!dot -Tpng tree.dot -o mass_tree.png
~~~

# 光谱数据建模

In [11]:
avgIRdata
labels_IR = [i[0] for i in avgIRdata.index.tolist()]

In [14]:
clf = DecisionTreeClassifier(random_state=2)
X = avgIRdata
#scores = cross_val_score(clf,X,Y,cv=4) 交叉验证
clf_ir = clf.fit(X,labels_IR)
feature_names = avgIRdata.columns.tolist()#list
dot_data = export_graphviz(clf_ir, out_file=None,
                          feature_names=feature_names,#list
                          class_names=clf.classes_,#list
                          filled=True, rounded=True,
                          special_characters=True)
graph = graphviz.Source(dot_data)

~~~
export_graphviz(clf_ir, out_file='tree.dot',
                          feature_names=feature_names,#list
                          class_names=clf.classes_,#list
                          filled=True, rounded=True,
                          special_characters=True)
!dot -Tpng tree.dot -o IR_tree.png
~~~

PCA+DecisionTreeClassifier

In [15]:
from sklearn.decomposition import PCA

In [72]:
pca = PCA(n_components=2,random_state=1).fit(X)

In [73]:
Xpca = pca.transform(X)

In [88]:
plt.rcParams['figure.figsize'] = (15,5)

In [94]:
labelA = ['not A' if l is not 'A' else l for l in labels_IR ]

In [85]:
clf_ir = clf.fit(Xpca,labels_IR)

In [79]:
dot_data = export_graphviz(clf_ir, out_file=None,
                          #feature_names=feature_names,#list
                          class_names=clf.classes_,#list
                          filled=True, rounded=True,
                          special_characters=True)
graph = graphviz.Source(dot_data)

array([ 0.72727273,  0.90909091,  1.        ])